In [ ]:
import csv
import numpy as np

## ETF Data Handling

In [6]:
import pandas as pd
import datetime

In [8]:
"""
# TODO: 
Get those etf name which exists before 12/31/2015
It should be:
    
etf_name_list = ['XLE', 'AMLP', 'VDE', 'AMJ', 'EMLP', 'XOP', 'MLPI', 'IXC', 'OIH', 'MLPA', 'IYE', 'MLPX', 'FENY', 'AMZA', 'ERX', 'ATMP', 'IEO', 'TPYP', 'AMU', 'TAN', 'GUSH', 'GREK', 'XES', 'RYE', 'FXN', 'ICLN', 'IEZ', 'PZD', 'PBW', 'DIG', 'FCG', 'MLPC', 'QCLN', 'GEX', 'FAN', 'KOL', 'ENFR', 'PXI', 'ZMLP', 'PBD', 'FILL', 'MLPY', 'DRIP', 'PXE', 'YMLP', 'CRAK', 'IMLP', 'GASL', 'MLPO', 'FRAK', 'PSCE', 'GRID', 'NLR', 'ERY', 'YMLI', 'PXJ', 'YLCO', 'PUW', 'DUG', 'ENY', 'MLPG', 'MLPB', 'AMUB', 'GASX', 'CHIE', 'DDG']
len(etf_name_list) == 66
    
try it !!
"""

etf_name_list = []
df = pd.read_csv("Energy Equity ETF List (80).csv")
date = { "name": df["Symbol"], "date": df["Inception"] }
date = pd.DataFrame(date)
#print(date.head())

#轉換時間格式
for a in range(0,len(date)):
    t = time.strptime(date["date"][a], "%d/%m/%Y")
    date["date"][a] = time.strftime("%Y-%m-%d", t)
#print(date.head()) 

#篩選日期
mask = date["date"] <= "2015-12-31"
etf_name_list = list(date[mask]["name"])
    
print(etf_name_list)
len(etf_name_list)

['XLE', 'AMLP', 'VDE', 'AMJ', 'EMLP', 'XOP', 'MLPI', 'IXC', 'OIH', 'MLPA', 'IYE', 'MLPX', 'FENY', 'AMZA', 'ERX', 'ATMP', 'IEO', 'TPYP', 'AMU', 'TAN', 'GUSH', 'GREK', 'XES', 'RYE', 'FXN', 'ICLN', 'IEZ', 'PZD', 'PBW', 'DIG', 'FCG', 'MLPC', 'QCLN', 'GEX', 'FAN', 'KOL', 'ENFR', 'PXI', 'ZMLP', 'PBD', 'FILL', 'MLPY', 'DRIP', 'PXE', 'YMLP', 'CRAK', 'IMLP', 'GASL', 'MLPO', 'FRAK', 'PSCE', 'GRID', 'NLR', 'ERY', 'YMLI', 'PXJ', 'YLCO', 'PUW', 'DUG', 'ENY', 'MLPG', 'MLPB', 'AMUB', 'GASX', 'CHIE', 'DDG']


66

## Day & Time Handling

In [ ]:
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
month_day_dict = {'Jan':31, 'Feb':28, 'Mar':31, 'Apr':30, 'May':31, 'Jun':30, 'Jul':31, 'Aug':31, 'Sep':30, 'Oct':31, 'Nov':30, 'Dec':31}
month_trans_dict = {'Jan':1, 'Feb':2, 'Mar':3, 'Apr':4, 'May':5, 'Jun':6, 'Jul':7, 'Aug':8, 'Sep':9, 'Oct':10, 'Nov':11, 'Dec':12}

last_day_of_stock_market_dict = {}

def is_last_day(time_list):
    '''
    In this function,
    Input a list = [month(int), day(int), yeat(int)]
    Output:
        True, if it is the last day of the month.
        False, if not.
    '''
    if time_list[2] == 2016 and time_list[0] == 'Feb':
        if time_list[1] == 29:
            return True
        else:
            return False
    else:
        if month_day_dict[time_list[0]] == time_list[1]:
            return True
        else:
            return False

def build_last_day_of_stock_market_dict():
    '''
    # TODO:
    We know that stock market closes at weekend. We need to know the exactly last day in stock market of a month.
    Build a dictionary that follows:
    
    last_day_dict['month(int)/day(int)/year(int)'] = True , if it is the last day of month in *stock market*.
                                                     False, if not.
                                                     
    where it only needs to handle between 12/31/2015(Thursday) ~ 12/31/2018(Monday)
    for example:
        last_day_dict['12/31/2015'] == True  , that is Thursday
        
        last_day_dict['12/29/2016'] == False , that is Tuesday
        last_day_dict['12/30/2016'] == True  , that is Friday
        last_day_dict['12/31/2016'] == False , that is Saturday, which stock market is closed.
        
    Read the calender carefully, and well-use every function and dictionary above.
    '''
    

## Web Crawler

In [ ]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import re

start_day = 1451404800 # Dec 30, 2015
end_day = 1546358400   # Mar 08, 2019
per_day = 86400

my_options = Options()
my_options.add_argument("--headless")
#my_options.add_argument("download.default_directory=.")
driver = webdriver.Chrome('./chromedriver.exe', chrome_options=my_options)
#driver = webdriver.Chrome('./chromedriver.exe')

def get_statistics(etf_name):
    '''
    In this function, we crawl the data of each etf by the name.
    Due to many complicating reasons, what I do in follow is:
        crawl from 12/31/2017 back to 12/31/2015,
        each iteration I only get 90 days, and use last_day_of_stock_market_dict to filter the valid days.
        records the needed NAV, and discards everything else.
    '''
    print('#### start handle %s ####'%(etf_name))
    statistic = []
    period2 = end_day
    period1 = end_day - per_day * 90
    while period1 >= start_day - per_day * 90:
        url = 'https://finance.yahoo.com/quote/%s/history?period1=%d&period2=%d&interval=1d&filter=history&frequency=1d'%(etf_name, period1, period2)
        driver.get(url)
        html = driver.page_source
        soup = BeautifulSoup(html, features='lxml')
        tr_list = soup.find_all('tr', {"class": "BdT Bdc($c-fuji-grey-c) Ta(end) Fz(s) Whs(nw)"})
        for tr in tr_list:
            td_time = tr.find('td', {'class': 'Py(10px) Ta(start) Pend(10px)'})
            time_list = re.split(r'[,\s]', td_time.get_text())
            
            del time_list[2]
            time_list[0] = month_trans_dict[time_list[0]]
            time_list[1] = int(time_list[1])
            time_list[2] = int(time_list[2])    #[month, day, year]
            
            if not last_day_of_stock_market_dict[ '%d/%d/%d'%(time_list[0], time_list[1], time_list[2]) ]:
                continue
            if '%d/%d/%d'%(time_list[0], time_list[1], time_list[2]) == '12/30/2015':
                break
                
            td_list = tr.find_all('td', {'class': 'Py(10px) Pstart(10px)'})
            statistic.append(float(td_list[3].get_text()))
        period2 = period1
        period1 = period2 - per_day * 90
    return statistic

def get_all_statistics(etf_name_list):
    '''
    In this function, we get the list of names of valid etf names.
    Return the nav records in the format:
    [
        [$1, $2, ...... , $37](for XLE),
        [$1, $2, ...... , $37](for AMLP),
            ...
        [$1, $2, ...... , $37](for DDG),
    ]
    '''
    statistics = []
    for etf_name in etf_name_list:
        statistic = get_statistics(etf_name)
        statistics.append(statistic)
    return statistics
    

## Main

In [ ]:
build_last_day_of_stock_market_dict()
statistics = get_all_statistics(etf_name_list)
print(statistics)